In [1]:
pwd

u'/Users/aurelperianu/Documents/Medicare_payments_classifier'

In [2]:
import pandas as pd
hosp_all=pd.read_csv('Medicare_Data/Hospital_Revised_Flatfiles_2014/HCAHPS - Hospital.csv')

In [3]:
hosp_all.shape

(147808, 17)

In [4]:
hosp_all.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Provider ID,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001,10001
Hospital Name,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER,SOUTHEAST ALABAMA MEDICAL CENTER
Address,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE,1108 ROSS CLARK CIRCLE
City,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN,DOTHAN
State,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL
ZIP Code,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301,36301
County Name,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON,HOUSTON
Phone Number,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701,3347938701
HCAHPS Measure ID,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_4_A_P,H_COMP_4_SN_P,H_COMP_4_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_6_N_P,H_COMP_6_Y_P
HCAHPS Question,Patients who reported that their room and bath...,Patients who reported that their room and bath...,Patients who reported that their room and bath...,"Patients who reported that their nurses ""Alway...","Patients who reported that their nurses ""Somet...","Patients who reported that their nurses ""Usual...","Patients who reported that their doctors ""Alwa...","Patients who reported that their doctors ""Some...","Patients who reported that their doctors ""Usua...","Patients who reported that they ""Always"" recei...","Patients who reported that they ""Sometimes"" or...","Patients who reported that they ""Usually"" rece...","Patients who reported that their pain was ""Alw...","Patients who reported that their pain was ""Som...","Patients who reported that their pain was ""Usu...","Patients who reported that staff ""Always"" expl...","Patients who reported that staff ""Sometimes"" o...","Patients who reported that staff ""Usually"" exp...","Patients who reported that NO, they were not g...","Patients who reported that YES, they were give..."


In [5]:
#hosp_all[hosp_all['provider_id']==130003]
#hosp_all[hosp_all['provider_id']==501338]


In [6]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
hosp_all=lower_columns(hosp_all)

In [7]:
hosp_all.columns.tolist()

['provider_id',
 'hospital_name',
 'address',
 'city',
 'state',
 'zip_code',
 'county_name',
 'phone_number',
 'hcahps_measure_id',
 'hcahps_question',
 'hcahps_answer_description',
 'hcahps_answer_percent',
 'number_of_completed_surveys',
 'survey_response_rate_percent',
 'footnote',
 'measure_start_date',
 'measure_end_date']

In [8]:
#drop columns
drop_col=['phone_number', 
 'hcahps_question',
 'number_of_completed_surveys',
 'survey_response_rate_percent',
 'footnote',
 'measure_start_date',
 'measure_end_date']
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx
hosp_all=drop_columns(drop_col, hosp_all)

In [9]:
#hosp_all.set_index(['provider_id', 'hcahps_measure_id'])
#hosp_all.reset_index()

In [10]:
#delete selective rows in 'hcahps_measure_id' column
drop_values=['H_CLEAN_','H_COMP_1','H_COMP_4','H_QUIET_HSP']
#df.drop(df.index[[1,3]], inplace=True)
for val in drop_values:
    hosp_all.drop(hosp_all[hosp_all['hcahps_measure_id'].str.startswith(val)].index, inplace=True)

In [11]:
hosp_all.shape

(92380, 10)

In [12]:
rating_ans=hosp_all[['hcahps_measure_id','hcahps_answer_description']].drop_duplicates()
#rating_ans

In [13]:
#change ratings from words to values
hosp_qual_val={'always':7,'usually':2,'never':-3,
               'No, staff':-3,'Yes, staff':7,
               'Strongly Agree':7,'Agree':2,'Disagree':-3,
               '(low)':-3,'(medium)':2,'(high)':7,
               'not recommend':-3,'would probably recommend':2,'would definitely recommend':7
              }

def rating_val(c):
    for key,value in hosp_qual_val.items():
        if key in c:
            return value

# df = pd.DataFrame(data={'portion':[1, 2, 3, 4], 'used':[1.0, 0.3, 0.0, 0.8]})

# df['alert'] = df.apply(alert, axis=1)
hosp_all['qual_val']=hosp_all['hcahps_answer_description'].apply(rating_val)

In [14]:
#hosp_all.fillna(float(0.01))
#calculate quality ratings for individual rows
hosp_all['hcahps_answer_percent']=hosp_all['hcahps_answer_percent'].apply(pd.to_numeric, errors='coerce')
hosp_all['row_rating']=hosp_all['qual_val']*hosp_all['hcahps_answer_percent']/float(100)

In [15]:
#hosp_all.fillna(float(0.01))

In [16]:
hosp_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92380 entries, 6 to 147807
Data columns (total 12 columns):
provider_id                  92380 non-null int64
hospital_name                92380 non-null object
address                      92380 non-null object
city                         92380 non-null object
state                        92380 non-null object
zip_code                     92380 non-null int64
county_name                  92300 non-null object
hcahps_measure_id            92380 non-null object
hcahps_answer_description    92380 non-null object
hcahps_answer_percent        81263 non-null float64
qual_val                     92380 non-null int64
row_rating                   81263 non-null float64
dtypes: float64(2), int64(3), object(7)
memory usage: 9.2+ MB


In [17]:
#hosp_all.head()

In [18]:
#
hosp_rating_gr=hosp_all.groupby(['provider_id','hospital_name','city','state','zip_code','county_name'])[['row_rating']].sum()
hosp_rating_gr=hosp_rating_gr.reset_index()
#hosp_rating_gr.fillna(1)

In [19]:
hosp_rating_gr.head()

,provider_id,hospital_name,city,state,zip_code,county_name,row_rating
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,HOUSTON,34.90
1,10005,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957,MARSHALL,37.95
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631,LAUDERDALE,31.80
3,10007,MIZELL MEMORIAL HOSPITAL,OPP,AL,36467,COVINGTON,36.60
4,10008,CRENSHAW COMMUNITY HOSPITAL,LUVERNE,AL,36049,CRENSHAW,35.35


In [20]:
#option to replace NaN rating with 0 (or 1) or to 
#hosp_rating_gr=hosp_rating_gr.replace(['nan','NaN'], ['.','0'])
hosp_rating_gr.fillna(0,inplace=True)
hosp_rating_gr = hosp_rating_gr[hosp_rating_gr.row_rating != 0]
hosp_rating_gr.sort_values(['row_rating'])

,provider_id,hospital_name,city,state,zip_code,county_name,row_rating
1912,220062,ADCARE HOSPITAL OF WORCESTER INC,WORCESTER,MA,1605,WORCESTER,9.20
3392,390025,KENSINGTON HOSPITAL,PHILADELPHIA,PA,19122,PHILADELPHIA,14.85
698,90008,UNITED MEDICAL CENTER,WASHINGTON,DC,20032,DISTRICT OF COLUMBIA,20.10
1098,140068,ROSELAND COMMUNITY HOSPITAL,CHICAGO,IL,60628,COOK,20.80
1145,140177,JACKSON PARK HOSPITAL,CHICAGO,IL,60649,COOK,21.40
499,50663,LOS ANGELES COMMUNITY HOSPITAL,LOS ANGELES,CA,90023,LOS ANGELES,21.80
2725,330027,NASSAU UNIVERSITY MEDICAL CENTER,EAST MEADOW,NY,11554,NASSAU,22.15
2420,270074,P H S INDIAN HOSPITAL AT BROWNING - BLACKFEET,BROWNING,MT,59417,GLACIER,22.50
4591,670058,EMERUS HOSPITAL,SUGAR LAND,TX,77479,FORT BEND,22.70
542,50763,SILVER LAKE MEDICAL CENTER,LOS ANGELES,CA,90026,LOS ANGELES,23.60


In [21]:
#hosp_rating_gr.dropna()
hosp_rating_gr.info()
hosp_rating_gr.sort_values(['row_rating'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4064 entries, 0 to 4610
Data columns (total 7 columns):
provider_id      4064 non-null int64
hospital_name    4064 non-null object
city             4064 non-null object
state            4064 non-null object
zip_code         4064 non-null int64
county_name      4064 non-null object
row_rating       4064 non-null float64
dtypes: float64(1), int64(2), object(4)
memory usage: 254.0+ KB


,provider_id,hospital_name,city,state,zip_code,county_name,row_rating
1912,220062,ADCARE HOSPITAL OF WORCESTER INC,WORCESTER,MA,1605,WORCESTER,9.20
3392,390025,KENSINGTON HOSPITAL,PHILADELPHIA,PA,19122,PHILADELPHIA,14.85
698,90008,UNITED MEDICAL CENTER,WASHINGTON,DC,20032,DISTRICT OF COLUMBIA,20.10
1098,140068,ROSELAND COMMUNITY HOSPITAL,CHICAGO,IL,60628,COOK,20.80
1145,140177,JACKSON PARK HOSPITAL,CHICAGO,IL,60649,COOK,21.40
499,50663,LOS ANGELES COMMUNITY HOSPITAL,LOS ANGELES,CA,90023,LOS ANGELES,21.80
2725,330027,NASSAU UNIVERSITY MEDICAL CENTER,EAST MEADOW,NY,11554,NASSAU,22.15
2420,270074,P H S INDIAN HOSPITAL AT BROWNING - BLACKFEET,BROWNING,MT,59417,GLACIER,22.50
4591,670058,EMERUS HOSPITAL,SUGAR LAND,TX,77479,FORT BEND,22.70
542,50763,SILVER LAKE MEDICAL CENTER,LOS ANGELES,CA,90026,LOS ANGELES,23.60


In [31]:
wa_hosp_rating_gr=hosp_rating_gr[hosp_rating_gr['state']=='WA']

In [29]:
#hosp_rating_gr_wa.sort_values(['row_rating']).head(70).T

,4286,4340,4284,4354,4314,4292,4283,4319,4289,4342,...,4282,4323,4280,4327,4333,4359,4322,4339,4355,4351
provider_id,500015,501316,500012,501330,500084,500026,500011,500129,500021,501318,...,500008,500150,500005,501302,501309,501335,500148,501315,501331,501327
hospital_name,MULTICARE AUBURN MEDICAL CENTER,KLICKITAT VALLEY HOSPITAL,YAKIMA REGIONAL MEDICAL AND CARDIAC CENTER,SUNNYSIDE COMMUNITY HOSPITAL,VALLEY GENERAL HOSPITAL,SWEDISH EDMONDS HOSPITAL,HIGHLINE MEDICAL CENTER,TACOMA GENERAL ALLENMORE HOSPITAL,ST CLARE HOSPITAL,OTHELLO COMMUNITY HOSPITAL,...,UNIVERSITY OF WASHINGTON MEDICAL CTR,LEGACY SALMON CREEK MEDICAL CENTER,VIRGINIA MASON MEDICAL CENTER,DAYTON GENERAL HOSPITAL,PROVIDENCE ST JOSEPH HOSPITAL,ST ELIZABETH HOSPITAL,CONFLUENCE HEALTH- WENATCHEE VALLEY HOSP & CLI...,SKYLINE HOSPITAL,PULLMAN REGIONAL HOSPITAL,WHITMAN HOSPITAL AND MEDICAL CENTER
city,AUBURN,GOLDENDALE,YAKIMA,SUNNYSIDE,MONROE,EDMONDS,BURIEN,TACOMA,LAKEWOOD,OTHELLO,...,SEATTLE,VANCOUVER,SEATTLE,DAYTON,CHEWELAH,ENUMCLAW,WENATCHEE,WHITE SALMON,PULLMAN,COLFAX
state,WA,WA,WA,WA,WA,WA,WA,WA,WA,WA,...,WA,WA,WA,WA,WA,WA,WA,WA,WA,WA
zip_code,98001,98620,98902,98944,98272,98026,98166,98415,98499,99344,...,98195,98686,98101,99328,99109,98022,98801,98672,99163,99111
county_name,KING,KLICKITAT,YAKIMA,YAKIMA,SNOHOMISH,SNOHOMISH,KING,PIERCE,PIERCE,ADAMS,...,KING,CLARK,KING,COLUMBIA,STEVENS,KING,CHELAN,KLICKITAT,WHITMAN,WHITMAN
row_rating,26.7,29.05,31.4,32,32.15,32.4,32.5,32.75,32.85,33.3,...,37.95,38.1,38.35,38.65,38.75,39.3,39.7,40.1,40.6,41.65


In [32]:
wa_hosp_rating_gr.to_pickle('wa_hosp_rating_gr_pickle.pkl') 
#wa_org.to_pickle('wa_org_pickle.pkl')

In [ ]:
wa_hosp_rating_gr= pd.read_pickle('wa_hosp_rating_gr_pickle.pkl')